In [3]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

In [4]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16,9]
plt.style.use('ggplot')
import os
import pandas as pd
import numpy as np 
import pickle as pk
import glob
from fbprophet import Prophet
import sys
sys.path.append('../')
import wiki
from wiki import utils 
import multiprocessing as mp
total_proc = None
from tqdm import tqdm
from IPython.display import clear_output

# Results Analysis
* ...

In [5]:
def load_plot(page, plot_y_org=True):
    forecast = pd.read_feather(PROPHET_PATH+VERSION+str(page)+'df.f')
    wiki.newphet.prophet_plot(forecast, plot_y_org=plot_y_org)
    plt.show()

In [36]:
PROPHET_PATH = '../data/prophet/'
RESULTS_PATH = 'results/'
BLENDS_PATH = 'blends/'
BLEND_VERSION = 'b1/'
#os.makedirs(PROPHET_PATH+BLENDS_PATH+BLEND_VERSION)

In [7]:
mediandf = pd.read_csv('../data/median_smapes.csv') 
mediandf = mediandf.iloc[:,[0,-1]]
mediandf.columns = ['page_index','median_smape']
mediandf.loc[:,'page_index'] = mediandf.page_index.astype(str)
mediandf.head()

,page_index,median_smape
0,0,40.582621
1,1,50.319311
2,2,49.209093
3,3,35.679932
4,4,58.634001


In [8]:
# black-list for model results that have incorrect smape (weren't using proper y_org)
black_list = ['v2.5df.f','v1.6df.f','v2df.f','v4df.f']
model_list = [x.split('/')[-1]  for x in glob.glob(PROPHET_PATH+RESULTS_PATH+'*') if x[-5] is not 'f']
model_list = [x for x in model_list if x not in black_list]
model_list

['v2.6df.f', 'v3df.f', 'v1.9df.f', 'v4.1df.f']

In [11]:
try:
    del df
except:
    pass
for file in model_list:
    tdf = pd.read_feather(PROPHET_PATH+RESULTS_PATH+file)
    try:
        df = df.merge(tdf[['page_index',file[:-4]+'_val']], on='page_index')
    except NameError:
        df = tdf[['page_index',file[:-4]+'_val']]

In [12]:
df = df.merge(mediandf, on='page_index')

In [13]:
df.head()

,page_index,v2.6_val,v3_val,v1.9_val,v4.1_val,median_smape
0,61780,21.464761,21.464761,143.016629,21.602433,27.617702
1,88262,100.961816,100.961816,114.985722,100.491853,48.228541
2,38562,33.040040,33.040040,35.051994,33.114415,57.389432
3,139999,64.424648,64.424648,160.727494,65.346381,91.585942
4,60766,60.384127,195.929825,198.711561,60.384127,57.434921


In [14]:
model_cols = df.iloc[:,1:].columns; model_cols

Index(['v2.6_val', 'v3_val', 'v1.9_val', 'v4.1_val', 'median_smape'], dtype='object')

In [15]:
len(df)

145063

In [16]:
df.iloc[:,1:].mean()

v2.6_val        56.867810
v3_val          58.629198
v1.9_val        81.696788
v4.1_val        57.112413
median_smape    47.541320
dtype: float64

pagedf = pd.read_feather(PROPHET_PATH+'pagedf.f')
ds = pd.read_feather(PROPHET_PATH+'ds.f')

page_counts = pagedf.count()
page_counts = page_counts.reset_index()
page_counts['index'] = page_counts['index'].astype(str)
page_counts.columns = ['page_index','data_count']; page_counts.tail()

In [17]:
df.iloc[:,1:].corr()

,v2.6_val,v3_val,v1.9_val,v4.1_val,median_smape
v2.6_val,1.000000,0.874333,0.626571,0.990572,0.602433
v3_val,0.874333,1.000000,0.623498,0.870509,0.570448
v1.9_val,0.626571,0.623498,1.000000,0.632000,0.381107
v4.1_val,0.990572,0.870509,0.632000,1.000000,0.596978
median_smape,0.602433,0.570448,0.381107,0.596978,1.000000


In [18]:
df.head()

,page_index,v2.6_val,v3_val,v1.9_val,v4.1_val,median_smape
0,61780,21.464761,21.464761,143.016629,21.602433,27.617702
1,88262,100.961816,100.961816,114.985722,100.491853,48.228541
2,38562,33.040040,33.040040,35.051994,33.114415,57.389432
3,139999,64.424648,64.424648,160.727494,65.346381,91.585942
4,60766,60.384127,195.929825,198.711561,60.384127,57.434921


In [19]:
df.iloc[:,1:].apply(min,axis=1).mean()

41.850809314256374

In [20]:
df.iloc[:,1:].apply(min,axis=1).head()

0    21.464761
1    48.228541
2    33.040040
3    64.424648
4    57.434921
dtype: float64

In [21]:
df.iloc[:,1:].idxmin(axis=1).head()

0        v2.6_val
1    median_smape
2        v2.6_val
3        v2.6_val
4    median_smape
dtype: object

In [22]:
df['min_val'] = df.iloc[:,1:].apply(min,axis=1)
df['min_model'] = df.iloc[:,1:].idxmin(axis=1)

In [23]:
df['min_model'].value_counts()

median_smape    73741
v2.6_val        27891
v4.1_val        22560
v1.9_val        15804
v3_val           3872
Name: min_model, dtype: int64

In [24]:
for col in model_cols:
    print(col, df[df.min_model == col][col].mean())

v2.6_val 41.0480226095
v3_val 60.1067001685
v1.9_val 57.4673014885
v4.1_val 36.9121301988
median_smape 39.3598920921


In [25]:
for col in model_cols:
    print('Without', col, df.loc[:,[x for x in model_cols if x != col]].apply(min,axis=1).mean())

Without v2.6_val 41.8902765779
Without v3_val 42.0766674029
Without v1.9_val 42.6177910736
Without v4.1_val 41.929438331
Without median_smape 52.9467568566


In [26]:
for col in model_cols:
    print('With only median and', col, df.loc[:,[col, 'median_smape']].apply(min,axis=1).mean())

With only median and v2.6_val 43.0932947583
With only median and v3_val 42.9978343451
With only median and v1.9_val 44.675683698
With only median and v4.1_val 43.1221820895
With only median and median_smape 47.5413203704


In [27]:
test_cols = ['v3_val', 'median_smape']

In [29]:
df['min_val'] = df.loc[:,test_cols].apply(min,axis=1)
df['min_model'] = df.loc[:,test_cols].idxmin(axis=1)

In [30]:
df.min_val.mean()

42.997834345113162

In [37]:
df[df.min_model == 'median_smape'].reset_index(
    ).loc[:,['page_index']].to_feather(PROPHET_PATH+BLENDS_PATH+BLEND_VERSION+'median'+'df.f')

In [38]:
df[df.min_model == 'v3_val'].reset_index(
    ).loc[:,['page_index']].to_feather(PROPHET_PATH+BLENDS_PATH+BLEND_VERSION+'v3'+'df.f')